# Top Model Configurations

**Import Packages**
The following cell will import the required packages, and print the their current version, and indicate how many GPU's are connected to the system.

In [44]:
# Author: Luke Collins
# Date Created: 2021-09-28
# Date Modified: 2021-09-28
# Description: This file contains the code used to run the experiments for the top model configuration experiments.

import os
import time
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn.metrics import confusion_matrix, roc_curve, f1_score, auc
from sklearn.metrics import precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import Xception, xception
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.regularizers import l1, l2, l1_l2

print("Printing versions of libraries used:")
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Seaborn:", sns.__version__)
print("Matplotlib:", mpl.__version__)
print("Scikit-learn version:", sk.__version__)
print("Tensorflow version:", tf.__version__)
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Printing versions of libraries used:
Numpy version: 1.23.5
Pandas version: 2.0.1
Seaborn: 0.12.2
Matplotlib: 3.7.1
Scikit-learn version: 1.2.2
Tensorflow version: 2.12.0
Number of GPUs Available:  1


**Notebook Configuration Cell**  
The following cell should always be run directly after the packages are imported, this is where you will configure directories to suit your system if reproducing this experiment at home. Do not proceed if your cells output produces and `false` for any of the directories.

In [45]:
# Load Directories
train_directory = "datasets/train-10"
test_directory = "datasets/test-10"
experiment_base_directory = "./experiments/top-model-configuration"

# Create directories if they don't exist
os.makedirs(experiment_base_directory, exist_ok=True)

# Check if directories exist
print("Train directory exists:", os.path.isdir(train_directory))
print("Test directory exists:", os.path.isdir(test_directory))
print("Experiment directory exists:", os.path.isdir(experiment_base_directory))

Train directory exists: True
Test directory exists: True
Experiment directory exists: True


**Dataset Creation Utilities**  
The following cell will define the functions required to create the training, validation and test datasets.

In [41]:
def create_train_val_datasets(
    input_dir,
    img_height=299,
    img_width=299,
    batch_size=32,
    augment_training_data=False
):
    """
    Creates train and validation datasets from images in the input directory.

    Args:
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        output_dir (str): The directory where the TFRecord files will be saved. Default is 'data/datasets'.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.
        save_dataset (bool): Whether to save the datasets as TFRecord files. Default is True.

    Returns:
        tf.data.Dataset, tf.data.Dataset: The train and validation datasets.
    """
    # Create ImageDataGenerator with validation split
    if augment_training_data:
        datagenTrain = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.2
                )
    else:
        datagenTrain = ImageDataGenerator(
            preprocessing_function=xception.preprocess_input,
            validation_split=0.2
        )
        
    datagenVal = ImageDataGenerator(
                preprocessing_function=xception.preprocess_input,
                validation_split=0.2
    )   

        
    
    # Create a train and validation data generator
    train_gen = datagenTrain.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="training",
        shuffle=True
    )

    val_gen = datagenVal.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="validation",
        shuffle=True
    )
    
    return train_gen, val_gen

def create_test_dataset(
    input_dir,
    img_height=299,
    img_width=299,
    batch_size=32,
):
    """
    Creates a test dataset from images in the input directory.

    Args:
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.

    Returns:
        tf.data.Dataset: The test dataset.
    """
    # Create ImageDataGenerator
    datagen = ImageDataGenerator(
        preprocessing_function=xception.preprocess_input
    )

    return datagen.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True,
    )

# Experiments
---
## Experiment #0001 | Layer Configuration Exploration
**Experiment ID:** 0001  
**Experiment Description:** Initial testing to select a top_model layer configuration.
  
**Experiment Outcome:** 
**Next Test:** 


In [72]:
base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.summary()

base_model.trainable = False

for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name:
        print(layer.name)

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_69 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_69[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [70]:
# Experiment ID:TMC-0001
experimentId = "TMC-0001"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=10  
)

lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_binary_accuracy', 
mode='max',
verbose=1,
save_best_only=True)


# Create a dataset and preprocess images to suit base model
train_ds, val_ds = create_train_val_datasets(experiment_train_directory, batch_size=32)
test_ds = create_test_dataset(experiment_test_directory, batch_size=32)


base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.trainable = False

for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name:
        layer.trainable = True


# Add a classification head (Top Model)
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(3070, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model) # Uppded Neurons to 3070 for 2048
top_model = Dropout(0.7)(top_model)  # Increased dropout rate
top_model = Dense(2048, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Changed Neruons to 2048 for 1024
top_model = Dropout(0.5)(top_model)
top_model = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Added bottom two layers
top_model = Dropout(0.2)(top_model)
top_model = Dense(1, activation = 'sigmoid')(top_model)

# Group the base_model and new fully-connected layers into a Model object
model = Model(inputs = base_model.input, outputs = top_model)

# Compile the model
model.compile(
    optimizer=Adam(1e-4),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

print(f"Test Accuracy: {evaluate[1]}")
print(f"Test Precision: {evaluate[2]}")

Found 19316 images belonging to 2 classes.
Found 4829 images belonging to 2 classes.
Found 4279 images belonging to 2 classes.
Epoch 1/50


2023-05-25 10:40:07.064308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


603/603 [==============================] - ETA: 0s - loss: 3.1590 - binary_accuracy: 0.7347 - precision_1: 0.7321 - recall_1: 0.7151

2023-05-25 10:41:24.175717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy improved from -inf to 0.75896, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 90s 142ms/step - loss: 3.1590 - binary_accuracy: 0.7347 - precision_1: 0.7321 - recall_1: 0.7151 - val_loss: 2.7383 - val_binary_accuracy: 0.7590 - val_precision_1: 0.8060 - val_recall_1: 0.6632 - lr: 1.0000e-04
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 1.9672 - binary_accuracy: 0.9350 - precision_1: 0.9308 - recall_1: 0.9357
Epoch 2: val_binary_accuracy did not improve from 0.75896
603/603 [==============================] - 84s 139ms/step - loss: 1.9672 - binary_accuracy: 0.9350 - precision_1: 0.9308 - recall_1: 0.9357 - val_loss: 2.0955 - val_binary_accuracy: 0.7429 - val_precision_1: 0.8142 - val_recall_1: 0.6083 - lr: 1.0000e-04
Epoch 3/50
603/603 [==============================] - ETA: 0s - loss: 1.2403 - binary_accuracy: 0.9711 - precision_1: 0.9692 - recall_1: 0.9713

2023-05-25 11:03:50.509709: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 11s 80ms/step - loss: 0.6839 - binary_accuracy: 0.8687 - precision_1: 0.8633 - recall_1: 0.8670
Test Accuracy: 0.8686560392379761
Test Precision: 0.8632643222808838


In [73]:
# Experiment ID:TMC-0001
experimentId = "TMC-0001"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=10  
)

lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_binary_accuracy', 
mode='max',
verbose=1,
save_best_only=True)


# Create a dataset and preprocess images to suit base model
train_ds, val_ds = create_train_val_datasets(experiment_train_directory, batch_size=32)
test_ds = create_test_dataset(experiment_test_directory, batch_size=32)


base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.trainable = False

for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name or "block11" in layer.name or "block10" in layer.name or "block9" in layer.name or "block8" in layer.name:
        layer.trainable = True


# Add a classification head (Top Model)
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(3070, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model) # Uppded Neurons to 3070 for 2048
top_model = Dropout(0.7)(top_model)  # Increased dropout rate
top_model = Dense(2048, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Changed Neruons to 2048 for 1024
top_model = Dropout(0.5)(top_model)
top_model = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Added bottom two layers
top_model = Dropout(0.2)(top_model)
top_model = Dense(1, activation = 'sigmoid')(top_model)

# Group the base_model and new fully-connected layers into a Model object
model = Model(inputs = base_model.input, outputs = top_model)

# Compile the model
model.compile(
    optimizer=Adam(1e-4),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

print(f"Test Accuracy: {evaluate[1]}")
print(f"Test Precision: {evaluate[2]}")

Found 19316 images belonging to 2 classes.
Found 4829 images belonging to 2 classes.
Found 4279 images belonging to 2 classes.
Epoch 1/50


2023-05-25 12:32:31.194192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


603/603 [==============================] - ETA: 0s - loss: 3.1015 - binary_accuracy: 0.8152 - precision_3: 0.8104 - recall_3: 0.8083

2023-05-25 12:34:21.562263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy improved from -inf to 0.81604, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 123s 192ms/step - loss: 3.1015 - binary_accuracy: 0.8152 - precision_3: 0.8104 - recall_3: 0.8083 - val_loss: 2.7621 - val_binary_accuracy: 0.8160 - val_precision_3: 0.8446 - val_recall_3: 0.7608 - lr: 1.0000e-04
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 2.0229 - binary_accuracy: 0.9614 - precision_3: 0.9590 - recall_3: 0.9616
Epoch 2: val_binary_accuracy improved from 0.81604 to 0.82583, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 116s 192ms/step - loss: 2.0229 - binary_accuracy: 0.9614 - precision_3: 0.9590 - recall_3: 0.9616 - val_loss: 2.1703 - val_binary_accuracy: 0.8258 - val_precision_3: 0.8654 - val_recall_3: 0.7591 - lr: 1.0000e-04
Epoch 3/50
603/603 [==============================] 

2023-05-25 13:11:14.482892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 11s 81ms/step - loss: 0.3946 - binary_accuracy: 0.9199 - precision_3: 0.9054 - recall_3: 0.9329
Test Accuracy: 0.919877827167511
Test Precision: 0.9053863883018494


In [76]:
# Experiment ID:TMC-0001
experimentId = "TMC-0001"

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-1", batch_size=32)
test_ds = create_test_dataset(experiment_test_directory, batch_size=32)

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=10  
)

lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_binary_accuracy', 
mode='max',
verbose=1,
save_best_only=True)


base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.trainable = False

# Add a classification head (Top Model)
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(3070, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model) # Uppded Neurons to 3070 for 2048
top_model = Dropout(0.7)(top_model) 
top_model = Dense(2048, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Changed Neruons to 2048 for 1024
top_model = Dropout(0.5)(top_model)
top_model = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Added bottom two layers
top_model = Dropout(0.2)(top_model)
top_model = Dense(1, activation = 'sigmoid')(top_model)

# Group the base_model and new fully-connected layers into a Model object
model = Model(inputs = base_model.input, outputs = top_model)

# Compile the model
model.compile(
    optimizer=Adam(1e-3),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate1 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)



for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name or "block11" in layer.name or "block10" in layer.name or "block9" in layer.name or "block8" in layer.name:
        layer.trainable = True
        
# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate2 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-2", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate3 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-3", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate4 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

print(f"Test Accuracy (Before): {evaluate1[1]}")
print(f"Test Precision (Before): {evaluate1[2]}")
print(f"Test Accuracy (Fine Tune): {evaluate2[1]}")
print(f"Test Precision (Fine Tune): {evaluate2[2]}")
print(f"Test Accuracy(Fine Fine Tune): {evaluate3[1]}")
print(f"Test Precision(Fine Fine Tune): {evaluate3[2]}")



Found 19316 images belonging to 2 classes.
Found 4828 images belonging to 2 classes.
Found 4279 images belonging to 2 classes.
Epoch 1/50


2023-05-25 14:25:53.074467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


603/603 [==============================] - ETA: 0s - loss: 1.5707 - binary_accuracy: 0.5206 - precision_6: 0.5068 - recall_6: 0.4630

2023-05-25 14:26:46.053893: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy improved from -inf to 0.54979, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 67s 107ms/step - loss: 1.5707 - binary_accuracy: 0.5206 - precision_6: 0.5068 - recall_6: 0.4630 - val_loss: 0.9258 - val_binary_accuracy: 0.5498 - val_precision_6: 0.5284 - val_recall_6: 0.6635 - lr: 0.0010
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 0.8202 - binary_accuracy: 0.5483 - precision_6: 0.5400 - recall_6: 0.4720
Epoch 2: val_binary_accuracy improved from 0.54979 to 0.57125, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 61s 101ms/step - loss: 0.8202 - binary_accuracy: 0.5483 - precision_6: 0.5400 - recall_6: 0.4720 - val_loss: 0.7579 - val_binary_accuracy: 0.5713 - val_precision_6: 0.5740 - val_recall_6: 0.4574 - lr: 0.0010
Epoch 3/50
603/603 [==============================] - ETA: 0s 

...

603/603 [==============================] - ETA: 0s - loss: 0.2966 - binary_accuracy: 0.8814 - precision_8: 0.9198 - recall_8: 0.8279

2023-05-25 15:11:50.959296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy improved from 0.73000 to 0.73312, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 124s 193ms/step - loss: 0.2966 - binary_accuracy: 0.8814 - precision_8: 0.9198 - recall_8: 0.8279 - val_loss: 0.7675 - val_binary_accuracy: 0.7331 - val_precision_8: 0.7813 - val_recall_8: 0.6246 - lr: 1.0000e-05
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 0.2312 - binary_accuracy: 0.9030 - precision_8: 0.9486 - recall_8: 0.8459
Epoch 2: val_binary_accuracy improved from 0.73312 to 0.73563, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 115s 191ms/step - loss: 0.2312 - binary_accuracy: 0.9030 - precision_8: 0.9486 - recall_8: 0.8459 - val_loss: 0.8462 - val_binary_accuracy: 0.7356 - val_precision_8: 0.8039 - val_recall_8: 0.6020 - lr: 1.0000e-05
Epoch 3/50
603/603 [=============================

2023-05-25 15:31:14.780396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 11s 79ms/step - loss: 0.6376 - binary_accuracy: 0.8438 - precision_8: 0.8478 - recall_8: 0.8269
Test Accuracy (Before): 0.6186560392379761
Test Precision (Before): 0.6312684416770935
Test Accuracy (Fine Tune): 0.8329417109489441
Test Precision (Fine Tune): 0.826756477355957
Test Accuracy(Fine Fine Tune): 0.84375
Test Precision(Fine Fine Tune): 0.8477937579154968


In [77]:
train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-3", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate4 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-4", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate5 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-5", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate6 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

Found 19318 images belonging to 2 classes.
Found 4828 images belonging to 2 classes.
Epoch 1/50


2023-05-25 15:51:39.648412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


603/603 [==============================] - ETA: 0s - loss: 0.2314 - binary_accuracy: 0.9059 - precision_9: 0.9533 - recall_9: 0.8477

2023-05-25 15:53:31.335010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy improved from 0.75396 to 0.76479, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 125s 196ms/step - loss: 0.2314 - binary_accuracy: 0.9059 - precision_9: 0.9533 - recall_9: 0.8477 - val_loss: 0.8145 - val_binary_accuracy: 0.7648 - val_precision_9: 0.8506 - val_recall_9: 0.6253 - lr: 1.0000e-05
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 0.1829 - binary_accuracy: 0.9189 - precision_9: 0.9702 - recall_9: 0.8592
Epoch 2: val_binary_accuracy improved from 0.76479 to 0.77417, saving model to ./experiments/top-model-configuration/TMC-0001/models/best_model.h5
603/603 [==============================] - 116s 192ms/step - loss: 0.1829 - binary_accuracy: 0.9189 - precision_9: 0.9702 - recall_9: 0.8592 - val_loss: 0.8261 - val_binary_accuracy: 0.7742 - val_precision_9: 0.8410 - val_recall_9: 0.6589 - lr: 1.0000e-05
Epoch 3/50
603/603 [=============================

...

603/603 [==============================] - ETA: 0s - loss: 0.1669 - binary_accuracy: 0.9261 - precision_11: 0.9576 - recall_11: 0.8871

2023-05-25 16:36:29.536906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy did not improve from 0.79500
603/603 [==============================] - 124s 193ms/step - loss: 0.1669 - binary_accuracy: 0.9261 - precision_11: 0.9576 - recall_11: 0.8871 - val_loss: 1.1319 - val_binary_accuracy: 0.7619 - val_precision_11: 0.8506 - val_recall_11: 0.6182 - lr: 1.0000e-05
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 0.1444 - binary_accuracy: 0.9304 - precision_11: 0.9628 - recall_11: 0.8911
Epoch 2: val_binary_accuracy did not improve from 0.79500
603/603 [==============================] - 115s 191ms/step - loss: 0.1444 - binary_accuracy: 0.9304 - precision_11: 0.9628 - recall_11: 0.8911 - val_loss: 1.1800 - val_binary_accuracy: 0.7558 - val_precision_11: 0.8605 - val_recall_11: 0.5917 - lr: 1.0000e-05
Epoch 3/50
603/603 [==============================] - ETA: 0s - loss: 0.1360 - binary_accuracy: 0.9351 - precision_11: 0.9669 - recall_11: 0.8969
Epoch 3: val_binary_accuracy did not improve from 0.79500
603/603 [====

2023-05-25 16:59:43.350460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 10s 78ms/step - loss: 0.7058 - binary_accuracy: 0.8682 - precision_11: 0.8697 - recall_11: 0.8575


In [79]:
print(f"Test Accuracy (Before): {evaluate4[1]}")
print(f"Test Precision (Before): {evaluate4[2]}")
print(f"Test Accuracy (Fine Tune): {evaluate5[1]}")
print(f"Test Precision (Fine Tune): {evaluate5[2]}")
print(f"Test Accuracy(Fine Fine Tune): {evaluate6[1]}")
print(f"Test Precision(Fine Fine Tune): {evaluate6[2]}")

Test Accuracy (Before): 0.8578477501869202
Test Precision (Before): 0.8483119606971741
Test Accuracy (Fine Tune): 0.862546980381012
Test Precision (Fine Tune): 0.8609756231307983
Test Accuracy(Fine Fine Tune): 0.8681861162185669
Test Precision(Fine Fine Tune): 0.8696717023849487


In [ ]:


# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

print(f"Test Accuracy: {evaluate[1]}")
print(f"Test Precision: {evaluate[2]}")


for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name or "block11" in layer.name or "block10" in layer.name or "block9" in layer.name or "block8" in layer.name:
        layer.trainable = True
        
# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

print(f"Test Accuracy: {evaluate[1]}")
print(f"Test Precision: {evaluate[2]}")

In [ ]:
# Experiment ID:TMC-0001
experimentId = "TMC-0001"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=3
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_binary_accuracy', 
mode='max',
verbose=1,
save_best_only=True)


# Create a dataset and preprocess images to suit base model
train_ds, val_ds = create_train_val_datasets(experiment_train_directory, batch_size=16)
test_ds = create_test_dataset(experiment_test_directory, batch_size=16)


base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.trainable = False

for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name:
        layer.trainable = True


# Add a classification head (Top Model)
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(3070, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model) # Uppded Neurons to 3070 for 2048
top_model = Dropout(0.7)(top_model)  # Increased dropout rate
top_model = Dense(2048, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Changed Neruons to 2048 for 1024
top_model = Dropout(0.5)(top_model)
top_model = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)  # Added bottom two layers
top_model = Dropout(0.2)(top_model)
top_model = Dense(1, activation = 'sigmoid')(top_model)

# Group the base_model and new fully-connected layers into a Model object
model = Model(inputs = base_model.input, outputs = top_model)

# Compile the model
model.compile(
    optimizer=Adam(1e-3),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)


# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)


# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

# Reset
train_ds.reset()
val_ds.reset()
test_ds.reset()


# Make predictions
train_preds = (model.predict(train_ds) > 0.5).astype("int32")
val_preds = (model.predict(val_ds) > 0.5).astype("int32")
test_preds = (model.predict(test_ds) > 0.5).astype("int32")

# Initialize metrics
train_precision_metric = tf.keras.metrics.Precision()
train_recall_metric = tf.keras.metrics.Recall()

val_precision_metric = tf.keras.metrics.Precision()
val_recall_metric = tf.keras.metrics.Recall()

test_precision_metric = tf.keras.metrics.Precision()
test_recall_metric = tf.keras.metrics.Recall()

# Update state of the metrics
train_precision_metric.update_state(train_ds.classes, train_preds)
train_recall_metric.update_state(train_ds.classes, train_preds)

val_precision_metric.update_state(val_ds.classes, val_preds)
val_recall_metric.update_state(val_ds.classes, val_preds)

test_precision_metric.update_state(test_ds.classes, test_preds)
test_recall_metric.update_state(test_ds.classes, test_preds)

# Calculate Performance Metrics
train_precision = train_precision_metric.result().numpy()
train_recall = train_recall_metric.result().numpy()
train_f1 = 2 * ((train_precision * train_recall) / (train_precision + train_recall))

val_precision = val_precision_metric.result().numpy()
val_recall = val_recall_metric.result().numpy()
val_f1 = 2 * ((val_precision * val_recall) / (val_precision + val_recall + 1e-5))

test_precision = test_precision_metric.result().numpy()
test_recall = test_recall_metric.result().numpy()
test_f1 = 2 * ((test_precision * test_recall) / (test_precision + test_recall + 1e-5))

# Compute confusion matrix for training set and test set
train_confusion_matrix = confusion_matrix(train_ds.classes, train_preds)
test_confusion_matrix = confusion_matrix(test_ds.classes, test_preds)

# Compute ROC curve for test set
test_fpr, test_tpr, _ = roc_curve(test_ds.classes, test_preds)
roc_auc = auc(test_fpr, test_tpr)

# Append results to accuracy_test list
accuracy_report.append({
    'processing_time': round(processing_time, 3),
    'test_accuracy': round(evaluate[1], 3),
    'test_loss': round(evaluate[0], 3),
    'test_precision': test_precision,
    'test_recall': test_recall,
    'test_f1': test_f1,
    'test_confusion_matrix': test_confusion_matrix,
    'test_roc_curve': (test_fpr, test_tpr),
    'test_roc_auc': roc_auc,
    'train_accuracy': round(history.history['binary_accuracy'][-1],3),
    'train_loss': round(history.history['loss'][-1],3),
    'train_precision': train_precision,
    'train_recall': train_recall,
    'train_f1': train_f1,
    'train_confusion_matrix': train_confusion_matrix,
    'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
    'val_loss': round(history.history['val_loss'][-1],3),
    'val_precision': val_precision,
    'val_recall': val_recall,
    'val_f1': val_f1,
})

for layer in base_model.layers:
    layers_report.append({
        'layer_name': layer.name,
        'layer_trainable': layer.trainable
    })
    

# Create DataFrame with accuracy report & layers report
accuracy_report_results = pd.DataFrame(accuracy_report)
layers_report_results = pd.DataFrame(layers_report)

# Save accuracy report & layers report to CSV
accuracy_report_results.to_csv(f'{experiment_results_directory}/accuracy_report_results.csv', index=False)
layers_report_results.to_csv(f'{experiment_results_directory}/layers_report_results.csv', index=False)

# Create and save confusion matrix plots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.heatmap(train_confusion_matrix, annot=True, fmt="d", ax=ax[0])
ax[0].set_title("Train Confusion Matrix")
sns.heatmap(test_confusion_matrix, annot=True, fmt="d", ax=ax[1])
ax[1].set_title("Test Confusion Matrix")
plt.savefig(f'{experiment_results_directory}/confusion_matrix.png')

# Create and save ROC curve plot
plt.figure(figsize=(8, 8))
plt.plot(test_fpr, test_tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig(f'{experiment_results_directory}/roc_curve.png')

In [68]:
# Experiment ID:TMC-0001
experimentId = "TMC-0001"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=3
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_binary_accuracy', 
mode='max',
verbose=1,
save_best_only=True)


# Create a dataset and preprocess images to suit base model
train_ds, val_ds = create_train_val_datasets(experiment_train_directory, batch_size=16)
test_ds = create_test_dataset(experiment_test_directory, batch_size=16)


base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.trainable = False

for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name:
        layer.trainable = True


# Add a classification head (Top Model)
top_model = base_model.outputmodel = keras.models.load_model('experiments/top-model-configuration/TMC-0001/models/best_model.h5')

# Reset the generators
train_ds.reset()
val_ds.reset()
test_ds.reset()
    
# Make predictions
# Make predictions
train_preds = (model.predict(train_ds) > 0.5).astype("int32")
Y_train_pred = model.predict(test_ds)
val_preds = (model.predict(val_ds) > 0.5).astype("int32")
test_preds = (model.predict(test_ds) > 0.5).astype("int32")

# Initialize metrics
train_precision_metric = tf.keras.metrics.Precision()
train_recall_metric = tf.keras.metrics.Recall()

val_precision_metric = tf.keras.metrics.Precision()
val_recall_metric = tf.keras.metrics.Recall()

test_precision_metric = tf.keras.metrics.Precision()
test_recall_metric = tf.keras.metrics.Recall()

# Update state of the metrics
train_precision_metric.update_state(train_ds.classes, train_preds)
train_recall_metric.update_state(train_ds.classes, train_preds)

val_precision_metric.update_state(val_ds.classes, val_preds)
val_recall_metric.update_state(val_ds.classes, val_preds)

test_precision_metric.update_state(test_ds.classes, test_preds)
test_recall_metric.update_state(test_ds.classes, test_preds)

# Calculate Performance Metrics
train_precision = train_precision_metric.result().numpy()
train_recall = train_recall_metric.result().numpy()
train_f1 = 2 * ((train_precision * train_recall) / (train_precision + train_recall))

val_precision = val_precision_metric.result().numpy()
val_recall = val_recall_metric.result().numpy()
val_f1 = 2 * ((val_precision * val_recall) / (val_precision + val_recall + 1e-5))

test_precision = test_precision_metric.result().numpy()
test_recall = test_recall_metric.result().numpy()
test_f1 = 2 * ((test_precision * test_recall) / (test_precision + test_recall + 1e-5))

# Compute confusion matrix for training set and test set
train_confusion_matrix = confusion_matrix(train_ds.classes, train_preds)
test_confusion_matrix = confusion_matrix(test_ds.classes, test_preds)

# Compute ROC curve for test set
test_fpr, test_tpr, _ = roc_curve(test_ds.classes, test_preds)
roc_auc = auc(test_fpr, test_tpr)

# Append results to accuracy_test list
accuracy_report.append({
    'processing_time': round(processing_time, 3),
    'test_accuracy': round(evaluate[1], 3),
    'test_loss': round(evaluate[0], 3),
    'test_precision': test_precision,
    'test_recall': test_recall,
    'test_f1': test_f1,
    'test_confusion_matrix': test_confusion_matrix,
    'test_roc_curve': (test_fpr, test_tpr),
    'test_roc_auc': roc_auc,
    'train_accuracy': round(history.history['binary_accuracy'][-1],3),
    'train_loss': round(history.history['loss'][-1],3),
    'train_precision': train_precision,
    'train_recall': train_recall,
    'train_f1': train_f1,
    'train_confusion_matrix': train_confusion_matrix,
    'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
    'val_loss': round(history.history['val_loss'][-1],3),
    'val_precision': val_precision,
    'val_recall': val_recall,
    'val_f1': val_f1,
})

for layer in base_model.layers:
    layers_report.append({
        'layer_name': layer.name,
        'layer_trainable': layer.trainable
    })
    

# Create DataFrame with accuracy report & layers report
accuracy_report_results = pd.DataFrame(accuracy_report)
layers_report_results = pd.DataFrame(layers_report)

# Save accuracy report & layers report to CSV
accuracy_report_results.to_csv(f'{experiment_results_directory}/accuracy_report_results.csv', index=False)
layers_report_results.to_csv(f'{experiment_results_directory}/layers_report_results.csv', index=False)

# Create and save confusion matrix plots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.heatmap(train_confusion_matrix, annot=True, fmt="d", ax=ax[0])
ax[0].set_title("Train Confusion Matrix")
sns.heatmap(test_confusion_matrix, annot=True, fmt="d", ax=ax[1])
ax[1].set_title("Test Confusion Matrix")
plt.savefig(f'{experiment_results_directory}/confusion_matrix.png')

# Create and save ROC curve plot
plt.figure(figsize=(8, 8))
plt.plot(test_fpr, test_tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig(f'{experiment_results_directory}/roc_curve.png')

# Group the base_model and new fully-connected layers into a Model object
model = Model(inputs = base_model.input, outputs = top_model)

# Compile the model
model.compile(
    optimizer=Adam(1e-3),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)


# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)


# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

# Reset
train_ds.reset()
val_ds.reset()
test_ds.reset()


# Make predictions
train_preds = (model.predict(train_ds) > 0.5).astype("int32")
val_preds = (model.predict(val_ds) > 0.5).astype("int32")
test_preds = (model.predict(test_ds) > 0.5).astype("int32")

# Initialize metrics
train_precision_metric = tf.keras.metrics.Precision()
train_recall_metric = tf.keras.metrics.Recall()

val_precision_metric = tf.keras.metrics.Precision()
val_recall_metric = tf.keras.metrics.Recall()

test_precision_metric = tf.keras.metrics.Precision()
test_recall_metric = tf.keras.metrics.Recall()

# Update state of the metrics
train_precision_metric.update_state(train_ds.classes, train_preds)
train_recall_metric.update_state(train_ds.classes, train_preds)

val_precision_metric.update_state(val_ds.classes, val_preds)
val_recall_metric.update_state(val_ds.classes, val_preds)

test_precision_metric.update_state(test_ds.classes, test_preds)
test_recall_metric.update_state(test_ds.classes, test_preds)

# Calculate Performance Metrics
train_precision = train_precision_metric.result().numpy()
train_recall = train_recall_metric.result().numpy()
train_f1 = 2 * ((train_precision * train_recall) / (train_precision + train_recall))

val_precision = val_precision_metric.result().numpy()
val_recall = val_recall_metric.result().numpy()
val_f1 = 2 * ((val_precision * val_recall) / (val_precision + val_recall + 1e-5))

test_precision = test_precision_metric.result().numpy()
test_recall = test_recall_metric.result().numpy()
test_f1 = 2 * ((test_precision * test_recall) / (test_precision + test_recall + 1e-5))

# Compute confusion matrix for training set and test set
train_confusion_matrix = confusion_matrix(train_ds.classes, train_preds)
test_confusion_matrix = confusion_matrix(test_ds.classes, test_preds)

# Compute ROC curve for test set
test_fpr, test_tpr, _ = roc_curve(test_ds.classes, test_preds)
roc_auc = auc(test_fpr, test_tpr)

# Append results to accuracy_test list
accuracy_report.append({
    'processing_time': round(processing_time, 3),
    'test_accuracy': round(evaluate[1], 3),
    'test_loss': round(evaluate[0], 3),
    'test_precision': test_precision,
    'test_recall': test_recall,
    'test_f1': test_f1,
    'test_confusion_matrix': test_confusion_matrix,
    'test_roc_curve': (test_fpr, test_tpr),
    'test_roc_auc': roc_auc,
    'train_accuracy': round(history.history['binary_accuracy'][-1],3),
    'train_loss': round(history.history['loss'][-1],3),
    'train_precision': train_precision,
    'train_recall': train_recall,
    'train_f1': train_f1,
    'train_confusion_matrix': train_confusion_matrix,
    'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
    'val_loss': round(history.history['val_loss'][-1],3),
    'val_precision': val_precision,
    'val_recall': val_recall,
    'val_f1': val_f1,
})

for layer in base_model.layers:
    layers_report.append({
        'layer_name': layer.name,
        'layer_trainable': layer.trainable
    })
    

# Create DataFrame with accuracy report & layers report
accuracy_report_results = pd.DataFrame(accuracy_report)
layers_report_results = pd.DataFrame(layers_report)

# Save accuracy report & layers report to CSV
accuracy_report_results.to_csv(f'{experiment_results_directory}/accuracy_report_results.csv', index=False)
layers_report_results.to_csv(f'{experiment_results_directory}/layers_report_results.csv', index=False)

# Create and save confusion matrix plots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.heatmap(train_confusion_matrix, annot=True, fmt="d", ax=ax[0])
ax[0].set_title("Train Confusion Matrix")
sns.heatmap(test_confusion_matrix, annot=True, fmt="d", ax=ax[1])
ax[1].set_title("Test Confusion Matrix")
plt.savefig(f'{experiment_results_directory}/confusion_matrix.png')

# Create and save ROC curve plot
plt.figure(figsize=(8, 8))
plt.plot(test_fpr, test_tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig(f'{experiment_results_directory}/roc_curve.png')

OSError: No file or directory found at experiments/top-model-configuration/TMC-0001/models/best_model.h5

In [ ]:
loss, binary_acc, precision, recall = model.evaluate(test_ds)
model = keras.models.load_model('experiments/top-model-configuration/TMC-0002/models/best_model.h5')

# Evaluate the model on the test dataset
loss, binary_acc, precision, recall = model.evaluate(test_ds)

# print("Test Loss:", test_loss)
# print("Test Accuracy:", test_accuracy)


In [ ]:

#Store results in variables
model_store = model
top_model_store = top_model
base_model_store = base_model
evaluate_store = evaluate
history_store = history

evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)


In [80]:
# Experiment ID:TMC-0005
experimentId = "TMC-0005"

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-1", batch_size=32)
test_ds = create_test_dataset(experiment_test_directory, batch_size=32)

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=10  
)

lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_binary_accuracy', 
mode='max',
verbose=1,
save_best_only=True)


base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3))
)

base_model.trainable = False

# Add a classification head (Top Model)
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(4096, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model)
top_model = Dropout(0.6)(top_model) 
top_model = Dense(2048, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model) 
top_model = Dropout(0.4)(top_model)
top_model = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(top_model) 
top_model = Dropout(0.3)(top_model)
top_model = Dense(1, activation = 'sigmoid')(top_model)

# Group the base_model and new fully-connected layers into a Model object
model = Model(inputs = base_model.input, outputs = top_model)

# Compile the model
model.compile(
    optimizer=Adam(0.001),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate1 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)



for layer in base_model.layers:
    if "block14" in layer.name or "block13" in layer.name or "block12" in layer.name or "block11" in layer.name or "block10" in layer.name or "block9" in layer.name or "block8" in layer.name:
        layer.trainable = True
        
train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-2", batch_size=32)
        
# Compile the model
model.compile(
    optimizer=Adam(1e-4),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate2 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-3", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-4),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate3 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

train_ds, val_ds = create_train_val_datasets("datasets/FF++/train-3", batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(1e-5),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)

# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)

# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate4 = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

print(f"Test Accuracy (Before): {evaluate1[1]}")
print(f"Test Precision (Before): {evaluate1[2]}")
print(f"Test Accuracy (Fine Tune): {evaluate2[1]}")
print(f"Test Precision (Fine Tune): {evaluate2[2]}")
print(f"Test Accuracy(Fine Fine Tune): {evaluate3[1]}")
print(f"Test Precision(Fine Fine Tune): {evaluate3[2]}")



Found 19316 images belonging to 2 classes.
Found 4828 images belonging to 2 classes.
Found 4279 images belonging to 2 classes.
Epoch 1/50


2023-05-25 17:11:06.423102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


603/603 [==============================] - ETA: 0s - loss: 1.5337 - binary_accuracy: 0.5437 - precision_12: 0.5330 - recall_12: 0.4846

2023-05-25 17:11:58.680394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_binary_accuracy improved from -inf to 0.55000, saving model to ./experiments/top-model-configuration/TMC-0005/models/best_model.h5
603/603 [==============================] - 65s 104ms/step - loss: 1.5337 - binary_accuracy: 0.5437 - precision_12: 0.5330 - recall_12: 0.4846 - val_loss: 0.9034 - val_binary_accuracy: 0.5500 - val_precision_12: 0.5642 - val_recall_12: 0.3154 - lr: 0.0010
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 0.8070 - binary_accuracy: 0.5871 - precision_12: 0.5826 - recall_12: 0.5268
Epoch 2: val_binary_accuracy improved from 0.55000 to 0.55792, saving model to ./experiments/top-model-configuration/TMC-0005/models/best_model.h5
603/603 [==============================] - 62s 103ms/step - loss: 0.8070 - binary_accuracy: 0.5871 - precision_12: 0.5826 - recall_12: 0.5268 - val_loss: 0.7671 - val_binary_accuracy: 0.5579 - val_precision_12: 0.6128 - val_recall_12: 0.2425 - lr: 0.0010
Epoch 3/50
603/603 [==============================] 

2023-05-25 17:30:11.936236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 11s 79ms/step - loss: 0.7275 - binary_accuracy: 0.6250 - precision_12: 0.6222 - recall_12: 0.5868
Found 19316 images belonging to 2 classes.
Found 4829 images belonging to 2 classes.
Epoch 1/50


2023-05-25 17:30:22.879983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-25 17:30:35.744869: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.50MiB (rounded to 13107200)requested by op model_46/batch_normalization_287/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-25 17:30:35.745070: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-05-25 17:30:35.745081: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 429, Chunks in use: 42

ResourceExhaustedError: Graph execution error:

Detected at node 'model_46/batch_normalization_287/FusedBatchNormV3' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/luke/Development/deepfake-detection/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/luke/Development/deepfake-detection/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/luke/Development/deepfake-detection/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/luke/Development/deepfake-detection/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
    